<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDA0321ENSkillsNetwork928-2023-01-01">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **JOBS API Implementation Using FLASK**



Estimated time needed: **45 to 60** minutes


## Objectives

You will be executing this code  so that the client application **Collecting Jobs API**  will be accessing this code executing on the server 


In [ ]:
!pip install flask
# %reset

## Dataset Used in this Assignment

The dataset used in this lab comes from the following source: https://www.kaggle.com/promptcloud/jobs-on-naukricom under the under a **Public Domain license**.

> Note: We are using a modified subset of that dataset for the lab, so to follow the lab instructions successfully please use the dataset provided with the lab, rather than the dataset from the original source.

The original dataset is a csv. We have converted the csv to json as per the requirement of the lab.


In [3]:
#%pip install wget
#%wget  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DA0321EN-SkillsNetwork/labs/module%201/Accessing%20Data%20Using%20APIs/jobs.json
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [3]:
import flask
from flask import request, jsonify
import requests
import re
import json
import datetime
import time
import pandas as pd

app = flask.Flask(__name__)

def get_data(key,value,current):
    print('get_data()')
    results = list()
    pattern_dict = {
        'C'      : '(C)',
        'C++'    : '(C\\+\\+)',
        'Java'   :'(Java)',
        'C#'     : '(C\\#)',
        'Python' :'(Python)',
        'Scala' : '(Scala)',
        'Oracle' : '(Oracle)',
        'SQL Server': '(SQL Server)',
        'MySQL Server' :'(MySQL Server)',
        'PostgreSQL':'(PostgreSQL)',
        'MongoDB'    : '(MongoDB)',
        'JavaScript'    : '(JavaScript)',
        'Los Angeles' :'(Los Angeles)',
        'New York':'(New York)',
        'San Francisco':'(San Francisco)',
        'Washington DC':'(Washington DC)',
        'Seattle':'(Seattle)',
        'Austin':'(Austin)',
        'Detroit':'(Detroit)',
        'technology':'(technology)'
        }
    for rec in current:
        print(rec[key])
        print(type(rec[key]))
        print(rec[key].find(value))
        #if rec[key].find(value) != -1:
        import re
        #reex_str = """(C)|(C\+\+)|(JavaScript)|(Java)|(C\#)|(Python)|(Scala)|(Oracle)|(SQL Server)|(MySQL Server)|(PostgreSQL)|(MongoDB)"""
        if re.search(pattern_dict[value],rec[key]) != None:
            results.append(rec)
    return results

@app.route('/', methods=['GET'])
def home():
    return '''<h1>Welcome to flask JOB search API</p>'''

@app.route('/data/all', methods=['GET'])
def api_all():
    print('api_all()')
    return jsonify(data)

@app.route('/data', methods=['GET'])
def api_id():
    # Check if keys such as Job Title,KeySkills, Role Category and others  are provided as part of the URL.
    #  Assign the keys to the corresponding variables..
    # If no key is provided, display an error in the browser.
    print('app route data api_id()')
    res = None
    for req in request.args:
        if req == 'Job Title':
            key = 'Job Title'
        elif req == 'Job Experience Required' :
            key='Job Experience Required'
        elif req == 'Key Skills' :
            key='Key Skills'
            
        elif req == 'Role Category' :
            key='Role Category'
        elif req == 'Location' :
            key='Location'
        
        elif req == 'Functional Area' :
            key='Functional Area'
        
        elif req == 'Industry' :
            key='Industry'
        elif req == 'Role' :
            key='Role'
        elif req=="id":
             key="id"
        else:
            #pass
            print('bad key in api_id() =', req)
    
        value = request.args[key]
        if (res==None):
            print('res is None from api_id(), get_data')
            res = get_data(key,value,data)
        else:
            print('res is not None from api_id()')
            res = get_data(key,value,res)

    # Use the jsonify function from Flask to convert our list of
    # Python dictionaries to the JSON format.
    return jsonify(res)   # note, not json, but response object, supposedly
print('done defs ', datetime.datetime.now())


done defs  2024-09-22 19:09:26.088770


In [116]:
## This is a slow kludgy way to do it.  Someday change
# it into a buffer of characters and write buffer to file
# when you get matching { " " : " " } - if single list, only look at 1st and last chars
# can use pop() on list or dict.  Could have { "key" : {subkey:subval,...
# so when get closing char, pop out until matching char, else push on stack
replist = [
        {r'\\r' : '' }, # remove carriage return left by DOS on MS Windows
        {r',[\\t \n]*\]' : r']' },  # get rid of comma before ] end of list, regardless of newlines
        {r',[\\t \n]*\}' : r'}' },  # get rid of comma before } end of {key:value}, regardless of newlines
        {r'[\\t \n]*\]$' : r']' },  # for last bracket, move it up to previous line
        # note that  [,  and  {,  could be valid syntax for empty first param BUT does it insert {{ ?
        # for function calls or array indices
        # but for json file, do fix
        {r'\[,' : '[' },
        {r'\{,' : '{' },
        {r'[\\t ]*\[[\\t ]*\n+'  : r'[' },  # forward to keep open bracket [ with text
        {r'[\\t ]*\{[\\t ]*\n+'  : r'\{' },  # debug1 forward to keep open brace { with text
        {r'\n+[\\t ]*\}' : r'}' },  # back to keep closing brace with text
        # what I would like to do is search for more than usual quotes and for that line sub all " to '
        # so then last 4 replacements will fix that. But also, what if uneven number like
        # "text1" : "text2 "Michelen's Guide" " },  or "text2 'comment for "some lang" "
        {r'\"' : "\'" },  # change all double quotes to singles and then next subs will fix
        {r'\'[\\t ]*:' : r'\"\:' },  # change single quote to double before :
        {r':[\\t ]*\'' : r'\:\"' },  # change single quote after :
        # below, what if we have {"Jane's book"    it's ok because only white space
        {r'\n+[\\t ]*\{[\\t ]*\'' : r'\n {"' },  #  debug quoteafterOpenCurlyl1'  change quote after newline and open line
        {r'\n+[\\t ]*\'' : r'\n "' }, # single quote at beginning of line
        {r'\'[\\t ]*\}' : r'"}' },  # change single quote before } WORKS but dels t
        {r'\'[\\t ]*,' : r'",' },  # change single quote comma newline WORKS but dels t
        {r'[\\t ]*\[[\\t ]*\{[\\t ]*\'' : r'[{"' },  ## another kludge - for 1st line
        {r'\\' : r'' }, # get rid of remaining backslashes
        {r'\n[^\[]' : r'\n    ' }]  # add indent to all but close bracket


In [118]:
# CURRENT  VERSION
# import warnings
# warnings.filterwarnings("ignore", category=FutureWarning)
data = None
import re

def cleanjsonfile():
    import re
    print('function cleanjsonfile() entered ', datetime.datetime.now())

    myfile = r'c:\cygwin64\home\krista\READABLE\jobs.json'
    print('entering open read()')
    with open(myfile, mode='r', encoding='utf-8') as f:
        longstring = f.read()
    print('type of longstring is ', type(longstring))
    print('len of longstring is ', len(longstring))
    # there are special characters and stuff in the file
    # and jsonify does not seem to work, so clean it up and write to a new file
    # below, white space or a newline is designated [\t \r] and * means any number of them
    # some assumptions: 1. the target string is a list using outer [] of dictionaries using inner {}
    #                   2. multi values use | not nested lists or key:value pairs, but i think nested stuff will be undisturbed
    #                   3. when  ,]  or  ,}  or {,  assume the comma is in error and remove it, but leave [, in case of filtering
    #                   4.  [  ]  {  } alone on a line are undesirable and json decode may not like them, so move forward or back
    #                   5.  ,[  ,{  alone on a line are undesirable, and json might not accept them
    #     \s  isn't working, so use [\t \r] and sometimes [\t \r\nprint(f'{longstring=} +2')
    # To remove debugging text use sed or ed and s/ + *'FIX.*'//g
    # delete whole line, don't leave white space or excess newline
    longstring1 = re.sub(r'\n[\\\\t \r\n]*\n' , r'\n' , longstring)  # get rid of lines with white space
    longstring = ("\n".join(item for item in longstring1.split('\n') if item))  # get rid of excess blank lines, stackoverflow
    print('len of longstring after remove blank lines = ', len(longstring))
    replacements = {}
    replacements = {k:v for e in replist for(k,v) in e.items()}  # turn list into dictionary
    # a safe way to do 1 regex at a time
    cnt = 0
    print('Beginning replacements, entering for loop')
    for nonjson, fixjson in replacements.items():
        cnt = cnt + 1
        #print('doing sub ', cnt)
        ##print('longstring slice ', longstring[longstring.find('Id'):longstring.find('}')+5] )
        # may need to pause between substitutions to allow time for each update sequentially
        # time.sleep(0.1) # Seconds
        longstring = re.sub(nonjson, fixjson, longstring)
    print('type of longstring is after for replacements ', type(longstring))
    print('len of longstring is after for replacements = ', len(longstring))

    with open(r"\cygwin64\home\krista\READABLE\newjobs.json", mode='w+', encoding='utf-8') as outy:
       nbrchars = outy.write(longstring)
       print('wrote the newjobs.json file')
import time
current_time = datetime.datetime.now()
print(current_time)


2024-09-22 20:01:08.362814


In [120]:
# %%timeit -n 1 -r 1   # later use n=250 and r=5 or something
current_time = datetime.datetime.now()
print('Wait until newjobs.json is written ', current_time)
cleanjsonfile()


Wait until newjobs.json is written  2024-09-22 20:01:10.195646
function cleanjsonfile() entered  2024-09-22 20:01:10.196643
entering open read()
type of longstring is  <class 'str'>
len of longstring is  12872418
len of longstring after remove blank lines =  12872417
Beginning replacements, entering for loop
type of longstring is after for replacements  <class 'str'>
len of longstring is after for replacements =  12791202
wrote the newjobs.json file


In [122]:
# Here is how to get the json decode error:
print('Open newjobs.json and do data = json.load(f) ', datetime.datetime.now()) 
myfile = r'c:\cygwin64\home\krista\READABLE\newjobs.json'
with open(myfile, mode='r', encoding='utf-8', errors=None) as f:
    data = json.load(f)
    print('after json.load(f) type of data is ', type(data))
print('json.load(f) complete ', datetime.datetime.now())
# JSONDecodeError: Expecting property name enclosed in double quotes: 
# line 59405 column 546 (char 3145330)
### deleted lines 59401-59409
# JSONDecodeError: Expecting property name enclosed in double quotes:
# line 114302 column 68 (char 6006091)
### deleted lines 114301-114309
# JSONDecodeError: Expecting property name enclosed in double quotes: 
# line 194657 column 1333 (char 10241779)
### deleted lines 194653-194661
# JSONDecodeError: Expecting property name enclosed in double quotes:
# line 228602 column 73 (char 12007657)
### deleted 228601,228609


Open newjobs.json and do data = json.load(f)  2024-09-22 20:02:26.831211
after json.load(f) type of data is  <class 'list'>
json.load(f) complete  2024-09-22 20:02:27.096423


In [124]:
app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [22/Sep/2024 20:02:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2024 20:02:51] "GET /data/all HTTP/1.1" 200 -


api_all()


127.0.0.1 - - [22/Sep/2024 20:04:20] "GET /data/all HTTP/1.1" 200 -


api_all()


## Authors


Lakshmi Holla  


### Other Contributors


Malika Singla  
Ayushi Jain


## Changelog


| Date (YYYY-MM-DD) | Version | Changed By        | Change Description                 |
| ----------------- | ------- | ----------------- | ---------------------------------- |
| 2022-01-03        | 0.1     | Lakshmi Holla  | Created Initial Version


Copyright © 2022 IBM Corporation. This notebook and its source code are released under the terms of the [MIT License](https://cognitiveclass.ai/mit-license?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDA0321ENSkillsNetwork21426264-2021-01-01&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBM-DA0321EN-SkillsNetwork-21426264&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ).
